In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import unicodedata
import re
import numpy as np
import os
import time
import pandas as pd
from sklearn.model_selection import train_test_split

from pylab import *
from matplotlib.font_manager import FontProperties
print('tf.__version__.is', tf.__version__)
print('tf.keras.__version__ is:', tf.keras.__version__)
print(tf.config.experimental.list_physical_devices('GPU'))

tf.__version__.is 2.5.0
tf.keras.__version__ is: 2.5.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
df = pd.read_csv('./preprocessed/train_matched_combined.csv')
df

,Original,Translated,Matched,Cosine_sim
0,耐吉 官方 男子 足球 長 褲 新款 標準 型 拒水 拉鏈 褲腳,Niji official men's football trousers new stan...,trousers,0.555
1,火影忍者 六道 鸣人 睡衣 卡卡 西宇智 波佐助 恤 二次元 动漫 短袖 衣服,Naruto Six Naruto Pajamas Caka Xiuxi Poto Asso...,flat sheet blanket cotton,0.000
2,小拇指 鞋坊 金標 貝殼頭 白紅時 尚 男女 鞋板 鞋 時尚 情侶 鞋 男鞋 女鞋,Small thumb shoes gold standard shell head whi...,fashion sports shoes women s shoes,0.784
3,居家 大 推款 記憶 棉枕護 勁 頸 椎 枕修 復 脊椎 治頸椎 專用 枕頭 女孕婦 失眠 ...,Home Big Push Memory Cotton Pillow Care Cervic...,flat sheet blanket cotton,0.000
4,炫彩 細口手 沖 壺桃紅 藍 黃,Colorful thin mouthful pot pot red blue yellow,mini pot extra small pot taho pot,0.526
...,...,...,...,...
135995,噴霧 瓶雪寧 稀釋 瓶 噴霧 瓶 有 刻度 家用 塑料瓶 小噴壺 廚房 洗潔 精分 裝瓶,Spray bottle Snow Ning dilute bottle spray bot...,bottle,0.791
135996,現貨 三角 造型 烤盤 造型 盤餐 盤,Spot triangle styling baking pan styling dishes,hair styling twist,0.385
135997,歐美韓 爆款韓 名媛 英倫學院 風搖滾 龐克 仿舊 性感 百搭 宜興 紫砂 杯純 全手工 茶...,"Europe and the United States, Korean explosive...",tea cup,0.639
135998,新款 字母 月亮 元素 珍珠 耳釘 別 緻 的 一款 經典 時尚,New alphabet moon elements Pearl earrings shou...,fashion earrings,0.408


In [3]:
df['Translated']

0         Niji official men's football trousers new stan...
1         Naruto Six Naruto Pajamas Caka Xiuxi Poto Asso...
2         Small thumb shoes gold standard shell head whi...
3         Home Big Push Memory Cotton Pillow Care Cervic...
4            Colorful thin mouthful pot pot red blue yellow
                                ...                        
135995    Spray bottle Snow Ning dilute bottle spray bot...
135996      Spot triangle styling baking pan styling dishes
135997    Europe and the United States, Korean explosive...
135998    New alphabet moon elements Pearl earrings shou...
135999                                      Passport holder
Name: Translated, Length: 136000, dtype: object

DROPPING NON-ENGLISH TRANSLATED ROWS (FAILED TRANSLATIONS BY GOOGLE API)

In [4]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [5]:
df['Translated'].apply(lambda x : isEnglish(x))

0         True
1         True
2         True
3         True
4         True
          ... 
135995    True
135996    True
135997    True
135998    True
135999    True
Name: Translated, Length: 136000, dtype: bool

In [6]:
df[~df['Translated'].apply(lambda x : isEnglish(x))]

,Original,Translated,Matched,Cosine_sim
42,現貨 正品 鞋運動 鞋彪 馬 女款 鞋子 亮面 時 尚款 蝴蝶 結鞋 休閒 鞋緞 帶 蝴蝶 ...,Spot genuine shoes sports shoes 彪 马 女 shoes hi...,shoes,0.809
85,閨蜜 淘 韓國代購 春夏 新款 老爹 鞋,闺 闺 淘 Korea purchasing spring and summer new o...,spring and summer new casual sports shoes wome...,0.480
135,炫客族 網紅 可 愛 帽子 拍照 頭套 搞怪 創意 角角 獨角獸 頭套 抖音 面具 搞笑 女,客 族 网 红 爱 拍 头 头 套 创 创 创 女 女 女 女 女 女 女 女 女 女 女 ...,flat sheet blanket cotton,0.000
138,菡萏 漸變 金邊 陶瓷 碗 餐具 吃 飯 碗碟 套裝 陶瓷 菜盤 魚盤子 家用 超取 請 聊...,菡萏 Gradient Bench Ceramic Bowl Tableware Eatin...,on chat many item,0.102
171,臻 品下 殺 小孩 兒童 眼罩 專用 幼兒園 夏季 遮光 學校 睡眠 午休 睡 卡通 可愛 ...,品 下 小 儿童 儿童 眼罩 幼 学 学 学 学 眠 宝 宝 学 学 学 学 学 学 学 学...,flat sheet blanket cotton,0.000
...,...,...,...,...
135910,宏興蓓朵 米色 單人座 沙發 台北 桃園 基隆市 區免 運費 新品,Hongxing 蓓 朵 米 单 三 人 沙发 台 基 市 市 市 区 区 新,flat sheet blanket cotton,0.000
135948,茶道 草編 榻榻米 墊 打坐 拜佛 墊 加厚 禪修 墊 禪 坐坐 墊藤編 蒲墊 蒲草 蒲團 ...,Tea Tao strap tatami mats to stay in worship B...,brake pad braker pad,0.550
135960,王蕎 淑,王 荞 淑,flat sheet blanket cotton,0.000
135977,現貨 四季 款 加厚 超順 滑大拜 墊 磕大頭 墊 拜佛 墊 拜 懺墊 大禮 拜 墊 家用,Spot four seasons thickened super smooth seaso...,brake pad braker pad,0.426


In [7]:
df.drop(df[~df['Translated'].apply(lambda x : isEnglish(x))].index, inplace = True)

In [8]:
df

,Original,Translated,Matched,Cosine_sim
0,耐吉 官方 男子 足球 長 褲 新款 標準 型 拒水 拉鏈 褲腳,Niji official men's football trousers new stan...,trousers,0.555
1,火影忍者 六道 鸣人 睡衣 卡卡 西宇智 波佐助 恤 二次元 动漫 短袖 衣服,Naruto Six Naruto Pajamas Caka Xiuxi Poto Asso...,flat sheet blanket cotton,0.000
2,小拇指 鞋坊 金標 貝殼頭 白紅時 尚 男女 鞋板 鞋 時尚 情侶 鞋 男鞋 女鞋,Small thumb shoes gold standard shell head whi...,fashion sports shoes women s shoes,0.784
3,居家 大 推款 記憶 棉枕護 勁 頸 椎 枕修 復 脊椎 治頸椎 專用 枕頭 女孕婦 失眠 ...,Home Big Push Memory Cotton Pillow Care Cervic...,flat sheet blanket cotton,0.000
4,炫彩 細口手 沖 壺桃紅 藍 黃,Colorful thin mouthful pot pot red blue yellow,mini pot extra small pot taho pot,0.526
...,...,...,...,...
135995,噴霧 瓶雪寧 稀釋 瓶 噴霧 瓶 有 刻度 家用 塑料瓶 小噴壺 廚房 洗潔 精分 裝瓶,Spray bottle Snow Ning dilute bottle spray bot...,bottle,0.791
135996,現貨 三角 造型 烤盤 造型 盤餐 盤,Spot triangle styling baking pan styling dishes,hair styling twist,0.385
135997,歐美韓 爆款韓 名媛 英倫學院 風搖滾 龐克 仿舊 性感 百搭 宜興 紫砂 杯純 全手工 茶...,"Europe and the United States, Korean explosive...",tea cup,0.639
135998,新款 字母 月亮 元素 珍珠 耳釘 別 緻 的 一款 經典 時尚,New alphabet moon elements Pearl earrings shou...,fashion earrings,0.408


Dataset loading

In [9]:
tcn_clean = df['Original'].reset_index(drop=True)
tcn_clean

0                          耐吉 官方 男子 足球 長 褲 新款 標準 型 拒水 拉鏈 褲腳
1                   火影忍者 六道 鸣人 睡衣 卡卡 西宇智 波佐助 恤 二次元 动漫 短袖 衣服
2                 小拇指 鞋坊 金標 貝殼頭 白紅時 尚 男女 鞋板 鞋 時尚 情侶 鞋 男鞋 女鞋
3         居家 大 推款 記憶 棉枕護 勁 頸 椎 枕修 復 脊椎 治頸椎 專用 枕頭 女孕婦 失眠 ...
4                                          炫彩 細口手 沖 壺桃紅 藍 黃
                                ...                        
130053         噴霧 瓶雪寧 稀釋 瓶 噴霧 瓶 有 刻度 家用 塑料瓶 小噴壺 廚房 洗潔 精分 裝瓶
130054                                  現貨 三角 造型 烤盤 造型 盤餐 盤
130055    歐美韓 爆款韓 名媛 英倫學院 風搖滾 龐克 仿舊 性感 百搭 宜興 紫砂 杯純 全手工 茶...
130056                     新款 字母 月亮 元素 珍珠 耳釘 別 緻 的 一款 經典 時尚
130057                                             護照 夾護 照套
Name: Original, Length: 130058, dtype: object

In [10]:
en_clean = df['Translated'].reset_index(drop=True)
en_clean

0         Niji official men's football trousers new stan...
1         Naruto Six Naruto Pajamas Caka Xiuxi Poto Asso...
2         Small thumb shoes gold standard shell head whi...
3         Home Big Push Memory Cotton Pillow Care Cervic...
4            Colorful thin mouthful pot pot red blue yellow
                                ...                        
130053    Spray bottle Snow Ning dilute bottle spray bot...
130054      Spot triangle styling baking pan styling dishes
130055    Europe and the United States, Korean explosive...
130056    New alphabet moon elements Pearl earrings shou...
130057                                      Passport holder
Name: Translated, Length: 130058, dtype: object

Preprocessing (change from unicode to ascii & add \<start\> and \<end\> tokens

In [11]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

In [12]:
def preprocess(w):
    w = unicode_to_ascii(w.lower().strip())

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [13]:
# u means unicode encoder
# en_sentence = en_clean['translation_output'][0]
# chn_sentence = tcn_clean['product_title_processed'][0]
en_sentence = en_clean[0]
chn_sentence = tcn_clean[0]
print(preprocess(en_sentence))
print(preprocess(chn_sentence))
print(preprocess(chn_sentence).encode('utf-8'))

<start> niji official men's football trousers new standard refusal zipper trousers <end>
<start> 耐吉 官方 男子 足球 長 褲 新款 標準 型 拒水 拉鏈 褲腳 <end>
b'<start> \xe8\x80\x90\xe5\x90\x89 \xe5\xae\x98\xe6\x96\xb9 \xe7\x94\xb7\xe5\xad\x90 \xe8\xb6\xb3\xe7\x90\x83 \xe9\x95\xb7 \xe8\xa4\xb2 \xe6\x96\xb0\xe6\xac\xbe \xe6\xa8\x99\xe6\xba\x96 \xe5\x9e\x8b \xe6\x8b\x92\xe6\xb0\xb4 \xe6\x8b\x89\xe9\x8f\x88 \xe8\xa4\xb2\xe8\x85\xb3 <end>'


Prepare Embedding Layer

In [14]:
def create_dataset(target, source):
    assert len(source) == len(target)
    word_pairs = [[preprocess(str(target[i])), preprocess(str(source[i]))]
                  for i in range(len(source))]

    # return two tuple: one tuple includes all English sentenses, and 
    # another tuple includes all Chinese sentenses
    return word_pairs

# word_pairs = create_dataset(en_clean['translation_output'], tcn_clean['product_title_processed'])
word_pairs = create_dataset(en_clean, tcn_clean)
word_pairs[:20]

[["<start> niji official men's football trousers new standard refusal zipper trousers <end>",
  '<start> 耐吉 官方 男子 足球 長 褲 新款 標準 型 拒水 拉鏈 褲腳 <end>'],
 ['<start> naruto six naruto pajamas caka xiuxi poto association second yuan anime short sleeve <end>',
  '<start> 火影忍者 六道 鸣人 睡衣 卡卡 西宇智 波佐助 恤 二次元 动漫 短袖 衣服 <end>'],
 ["<start> small thumb shoes gold standard shell head white red fashion men and women shoe shoes fashion couple shoes men's shoes women's shoes <end>",
  '<start> 小拇指 鞋坊 金標 貝殼頭 白紅時 尚 男女 鞋板 鞋 時尚 情侶 鞋 男鞋 女鞋 <end>'],
 ['<start> home big push memory cotton pillow care cervic pillow repair spine cervical pillow female pregnant women insomnia health physical treatment pillow <end>',
  '<start> 居家 大 推款 記憶 棉枕護 勁 頸 椎 枕修 復 脊椎 治頸椎 專用 枕頭 女孕婦 失眠 保健 理療 枕 <end>'],
 ['<start> colorful thin mouthful pot pot red blue yellow <end>',
  '<start> 炫彩 細口手 沖 壺桃紅 藍 黃 <end>'],
 ['<start> personality irregular splicing rope design round neck short-sleeved shirt <end>',
  '<start> 個性 不 規則 拼接 抽繩 設計 圓領 短袖 恤 <en

In [15]:
# en, chn = zip(*create_dataset(en_clean['translation_output'], tcn_clean['product_title_processed']))
en, chn = zip(*create_dataset(en_clean, tcn_clean))
print(en[-1])
print(chn[-1])
# show the size of the dataset
assert len(en) == len(chn)
print("Size:", len(en))

<start> passport holder <end>
<start> 護照 夾護 照套 <end>
Size: 130058


In [16]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='')
    # generate a dictionary, e.g. word -> index(of the dictionary)
    lang_tokenizer.fit_on_texts(lang)

    # output the vector sequences, e.g. [1, 7, 237, 3, 2]
    tensor = lang_tokenizer.texts_to_sequences(lang)

    # padding sentences to the same length
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                           padding='post')
    return tensor, lang_tokenizer

In [17]:
def load_dataset(target, source):
    # creating cleaned input, output pairs
    # regard Chinese as source sentence, regard English as target sentence
    targ_lang, inp_lang = zip(*create_dataset(target, source))

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [18]:
def max_length(tensor):
    # padding the sentence to max_length
    return max(len(t) for t in tensor)

In [19]:
# input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(en_clean['translation_output'], 
#                                                                 tcn_clean['product_title_processed'])
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(en_clean, 
                                                                tcn_clean)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(
    target_tensor), max_length(input_tensor)

# Creating training and validation sets using an 95-5 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(
    input_tensor, target_tensor, test_size=0.05)

# Show length of the training data and validation data
print("# training data: {:d}\n# test data: {:d}".format(len(input_tensor_train), len(input_tensor_val)))

# training data: 123555
# test data: 6503


In [20]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print("%d ----> %s" % (t, lang.index_word[t]))

print("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print()
print("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
1598 ----> 米色
25 ----> 休閒
31 ----> 運動
118720 ----> 鞋朴
118721 ----> 敘俊著用
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
1623 ----> beige
26 ----> casual
20 ----> sports
3 ----> shoes
19 ----> simple
16751 ----> narrative
2 ----> <end>


Initialise Parameters 

In [21]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 128
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 32
units = 32
# 0 is a reserved index that won't be assigned to any word, so the size of vocabulary should add 1
vocab_inp_size = len(inp_lang.word_index) + 1
vocab_tar_size = len(targ_lang.word_index) + 1

dataset = tf.data.Dataset.from_tensor_slices(
    (input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([128, 90]), TensorShape([128, 125]))

In [22]:
print('vocab_inp_size =', vocab_inp_size)
print('vocab_tar_size =', vocab_tar_size)

vocab_inp_size = 150586
vocab_tar_size = 50550


Encoder class construction

In [23]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        # vacab_size=vocab_inp_size=150586, embedding_dim=32 enc_units=32 batch_sz=128
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_activation='sigmoid',
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        # x is the training data with shape == (batch_size, max_length)  -> (128, 90)
        # which means there are batch_size sentences in one batch, the length of each sentence is max_length
        # hidden state shape == (batch_size, units) -> (128, 32)
        # after embedding, x shape == (batch_size, max_length, embedding_dim) -> (128, 90, 32)
        x = self.embedding(x)
        
        # output contains the state(in GRU, the hidden state and the output are same) from all timestamps,
        # output shape == (batch_size, max_length, units) -> (128, 90, 32)
        # state is the hidden state of the last timestamp, shape == (batch_size, units) -> (128, 32)
        output, state = self.gru(x, initial_state=hidden)
        
        return output, state

    def initialize_hidden_state(self):
        # initialize the first state of the gru,  shape == (batch_size, units) -> (128, 32)
        return tf.zeros((self.batch_sz, self.enc_units))

In [24]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

# the output and the hidden state of GRU is equal
print(sample_output[-1, -1, :] == sample_hidden[-1, :])

Encoder output shape: (batch size, sequence length, units) (128, 90, 32)
Encoder Hidden state shape: (batch size, units) (128, 32)
tf.Tensor(
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True], shape=(32,), dtype=bool)


Define Attention layer

In [25]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape == (batch_size, max_length, hidden_size)
        context_vector = attention_weights * values
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights


In [26]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (128, 32)
Attention weights shape: (batch_size, sequence_length, 1) (128, 90, 1)


Decoder class construction

In [27]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        # vocab_size=vocab_tar_size=50550, embedding_dim=32, dec_units=32, batch_sz=128
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        # the dimension of the output is the vocab size, through the softmax function,
        # this layer will return the probability of each word in the dictory
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # This function outputs a result at each timestamp
        # The hidden state of fisrt timestamp in the decoder is 
        # the hidden state of last timestamp in the encoder
        context_vector, attention_weights = self.attention(hidden, enc_output)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # concatenate the input x and the context_vector, as the input of the GRU
        # context_vector shape == (batch_size, units) -> (128, 32)
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size) -> (128, 1, 32 + 32)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        # get the output and state of the current timestamp
        # output shape == (batch_size, 1, units) -> (128, 1, 32) 
        # state shape == (batch_size, units) -> (128, 32)
        output, state = self.gru(x)

        # output shape == (batch_size, hidden_size) -> (128, 32)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab) -> (128, 50550)
        x = self.fc(output)

        return x, state, attention_weights

In [28]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_output)
print('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (128, 50550)


Define optimizer and loss function

In [29]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    """Calculate the loss value
    Args:
        real: the true label  shape == (batch_size,) -> (128,)
        pred: the probability of each word from the vocabulary, is the output from the decoder 
                 shape == (batch_size, vocab_size) -> (128, 6082)

    Returns: 
        the average loss of the data in a batch size
    """
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

Checkpoints saving

In [30]:
checkpoint_dir = './checkpoints/tcn-eng1'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

Define Training

In [31]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden
        
        # feed the <start> as the first input of the decoder
        # dec input shape == (batch_size, 1) -> (128, 1)
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        # because of the data preprocessing(add a start token to the sentence)
        # the first word is <start>, so t starts from 1(not 0)
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            # targ[:, t] is the true label(index of the word) of every sentence(in a batch) 
            # at the current timestamp
            # like [  85   18   25   25  ···  1047   79   13], shape == (batch_size,) -> (128,)
            # predictions shape == (batch_size, vocab_size) -> (128, 6082)
            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    # collect all trainable variables
    variables = encoder.trainable_variables + decoder.trainable_variables

    # calculate the gradients for the whole variables
    gradients = tape.gradient(loss, variables)

    # apply the gradients on the variables
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [32]:
EPOCHS = 100

for epoch in range(EPOCHS):
    start = time.time()

    # get the initial hidden state of gru
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        checkpoint.save(file_prefix=checkpoint_prefix)
        
    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.1690
Epoch 1 Batch 100 Loss 0.8556
Epoch 1 Batch 200 Loss 0.8413
Epoch 1 Batch 300 Loss 0.7656
Epoch 1 Batch 400 Loss 0.8065
Epoch 1 Batch 500 Loss 0.8488
Epoch 1 Batch 600 Loss 0.8311
Epoch 1 Batch 700 Loss 0.8248
Epoch 1 Batch 800 Loss 0.7563
Epoch 1 Batch 900 Loss 0.8780
Epoch 1 Loss 0.8370
Time taken for 1 epoch 541.8934578895569 sec

Epoch 2 Batch 0 Loss 0.8217
Epoch 2 Batch 100 Loss 0.8037
Epoch 2 Batch 200 Loss 0.7415
Epoch 2 Batch 300 Loss 0.7504
Epoch 2 Batch 400 Loss 0.8557
Epoch 2 Batch 500 Loss 0.8075
Epoch 2 Batch 600 Loss 0.7624
Epoch 2 Batch 700 Loss 0.8408
Epoch 2 Batch 800 Loss 0.7458
Epoch 2 Batch 900 Loss 0.7746
Epoch 2 Loss 0.7786
Time taken for 1 epoch 462.73222255706787 sec

Epoch 3 Batch 0 Loss 0.7767
Epoch 3 Batch 100 Loss 0.7412
Epoch 3 Batch 200 Loss 0.7741
Epoch 3 Batch 300 Loss 0.7012
Epoch 3 Batch 400 Loss 0.7114
Epoch 3 Batch 500 Loss 0.7144
Epoch 3 Batch 600 Loss 0.6772
Epoch 3 Batch 700 Loss 0.7017
Epoch 3 Batch 800 Loss 0.6907
Epo

Evaluation (Translation)

In [31]:
def evaluate(sentence):
    """Translate a sentence
    Args:
        sentence: the test sentence        
    """
    
    # max_length_targ 38, max_length_inp 64
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess(sentence)

    # convert each word to the index in the test sentence
    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')

    inputs = tf.convert_to_tensor(inputs)

    result = ''

    # hidden shape == (1, 1024)
    hidden = [tf.zeros((1, units))]
    
    # enc out shape == (1, max_length_inp, 1024) -> (1, 18, 1024)
    # enc hidden shape == (1, 1024)
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        # get the index which has the highest probability
        predicted_id = tf.argmax(predictions[0]).numpy()
        #  convert the index to the word
        result += targ_lang.index_word[predicted_id] + ' '

        # when the decoder predicts the end, stop prediction
        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted id is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot


def translate(sentence):
    result, sentence, attention_plot= evaluate(sentence)
    return result.rpartition('<end>')[0].strip()


Restore the latest checkpoint and translate test set

In [32]:
checkpoint_dir = './checkpoints/tcn-eng1'
print(tf.train.latest_checkpoint(checkpoint_dir))

# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

./checkpoints/tcn-eng1\ckpt-20


In [33]:
test_set = pd.read_csv('./preprocessed/cleaned_test_tcn.csv')
test_set

,text,split,product_title_clean,product_title_processed
0,【PolarStar】美麗諾羊毛保暖襪『淺灰』P18634,public,美麗諾羊毛保暖襪淺灰,美麗諾 羊毛 保暖 襪淺 灰
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,public,甜蜜水晶天然水晶五行珠手鍊手鍊,甜蜜 水晶 天然 水晶 五行 珠手 鍊 手 鍊
2,粉晶六角柱純銀項鍊,public,粉晶六角柱純銀項鍊,粉晶 六角 柱純 銀項 鍊
3,3M SCOTCH VHB 超強力雙面膠-戶外專用 V1808,public,超強力雙面膠戶外專用,超強力 雙面 膠戶 外 專用
4,燈專屬優惠 *4盒,public,燈專屬優惠盒,燈 專屬 優惠盒
...,...,...,...,...
9995,RAINS BACKPACK 後背包 公事包 筆電包 丹麥品牌 白色,private,後背包公事包筆電包丹麥品牌白色,後 背包 公事包 筆電 包丹麥 品牌 白色
9996,Airwalk 凉鞋 童鞋 深蓝色 中童 A823230180 no002,private,凉鞋童鞋深蓝色中童,凉鞋 童鞋 深蓝色 中童
9997,NORNS 【迪士尼5000mAh行動電源(泰瑞鴨)】迪士尼 鴨嘴獸泰瑞 充電,private,迪士尼行動電源泰瑞鴨迪士尼鴨嘴獸泰瑞充電,迪士尼 行動 電源 泰瑞 鴨 迪士尼 鴨 嘴獸 泰瑞 充電
9998,"DERWENT 達爾文設計製圖鉛筆 鐵盒12支(6B-4H),設計*34214",private,達爾文設計製圖鉛筆鐵盒支設計,達爾文設 計 製 圖 鉛 筆 鐵 盒 支 設 計


In [34]:
test_tr_sents_all = []
for i in range(len(test_set['product_title_processed'])):
    try: test_tr_sents_all.append(translate(test_set['product_title_processed'][i]))
    except: test_tr_sents_all.append(np.nan)
        

In [35]:
test_tr_sents_all

[nan,
 'sweet crystal natural crystal five golden hand chain',
 nan,
 nan,
 nan,
 'special promotional models niji sports casual shoes female',
 'gift box of metal attached has a',
 'natural license board shoe walnut snow',
 'sandals black red men',
 'new zealand olive flower natural massage oil',
 nan,
 'japanese disney mickey minnie mickey carry-cut scissors portable scissors storage scissors scissors',
 nan,
 'australian papaya pushing shower',
 'fashion temperament fresh practical hair band hair accessories headband',
 nan,
 nan,
 nan,
 nan,
 'switch with safety safe iron paper frame wall wine home lucky',
 'banana styling warm cow fluorescent',
 'international brand',
 'lock buckle storage box',
 'that the burning wax out of plastic plastic mud',
 'the antibacterial stainless steel group with net bag',
 nan,
 'classic stimulating hooded jacket',
 nan,
 nan,
 'korean cream eye shadow face multiple optional',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'package hip chiffon fish tail 

In [36]:
test_set['prediction'] = test_tr_sents_all

In [37]:
test_set

,text,split,product_title_clean,product_title_processed,prediction
0,【PolarStar】美麗諾羊毛保暖襪『淺灰』P18634,public,美麗諾羊毛保暖襪淺灰,美麗諾 羊毛 保暖 襪淺 灰,NaN
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,public,甜蜜水晶天然水晶五行珠手鍊手鍊,甜蜜 水晶 天然 水晶 五行 珠手 鍊 手 鍊,sweet crystal natural crystal five golden hand...
2,粉晶六角柱純銀項鍊,public,粉晶六角柱純銀項鍊,粉晶 六角 柱純 銀項 鍊,NaN
3,3M SCOTCH VHB 超強力雙面膠-戶外專用 V1808,public,超強力雙面膠戶外專用,超強力 雙面 膠戶 外 專用,NaN
4,燈專屬優惠 *4盒,public,燈專屬優惠盒,燈 專屬 優惠盒,NaN
...,...,...,...,...,...
9995,RAINS BACKPACK 後背包 公事包 筆電包 丹麥品牌 白色,private,後背包公事包筆電包丹麥品牌白色,後 背包 公事包 筆電 包丹麥 品牌 白色,NaN
9996,Airwalk 凉鞋 童鞋 深蓝色 中童 A823230180 no002,private,凉鞋童鞋深蓝色中童,凉鞋 童鞋 深蓝色 中童,NaN
9997,NORNS 【迪士尼5000mAh行動電源(泰瑞鴨)】迪士尼 鴨嘴獸泰瑞 充電,private,迪士尼行動電源泰瑞鴨迪士尼鴨嘴獸泰瑞充電,迪士尼 行動 電源 泰瑞 鴨 迪士尼 鴨 嘴獸 泰瑞 充電,NaN
9998,"DERWENT 達爾文設計製圖鉛筆 鐵盒12支(6B-4H),設計*34214",private,達爾文設計製圖鉛筆鐵盒支設計,達爾文設 計 製 圖 鉛 筆 鐵 盒 支 設 計,NaN


In [38]:
test_set.isna().sum()

text                          0
split                         0
product_title_clean          72
product_title_processed      72
prediction                 5306
dtype: int64

In [39]:
test_set.dropna(inplace=True)
test_set

,text,split,product_title_clean,product_title_processed,prediction
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,public,甜蜜水晶天然水晶五行珠手鍊手鍊,甜蜜 水晶 天然 水晶 五行 珠手 鍊 手 鍊,sweet crystal natural crystal five golden hand...
5,特價促銷款 NIKE 耐吉 WMNS NIKE INTERNATIONALIST JCRD ...,public,特價促銷款耐吉運動休閒鞋女,特價 促銷款 耐吉運動 休閒鞋 女,special promotional models niji sports casual ...
6,Jo Malone 禮盒(附上緞帶),public,禮盒附上緞帶,禮盒 附上 緞帶,gift box of metal attached has a
7,【TORO】自然風2.7尺鞋櫃 (HX-926(胡桃) HX927雪白,public,自然風尺鞋櫃胡桃雪白,自然 風尺 鞋櫃 胡桃 雪白,natural license board shoe walnut snow
8,G.P 涼鞋 黑紅 男款 no686,public,涼鞋黑紅男款,涼鞋 黑紅 男款,sandals black red men
...,...,...,...,...,...
9986,超值2入 多功能汽車遮陽板眼鏡夾 顏色隨機,private,超值入多功能汽車遮陽板眼鏡夾顏色隨機,超值 入 多功能 汽車 遮陽 板眼 鏡 夾顏色 隨機,value into multi-function car sunshade eyeglas...
9987,金四季寵物窩,private,金四季寵物窩,金 四季 寵物 窩,golden four leaf net fluorescent
9989,針織外套 單排扣開衫外套★ifairies【41041】,private,針織外套單排扣開衫外套,針織 外套 單排 扣 開衫 外套,knitted jacket single-row buckle jacket
9990,超大棉麻衣物雙開收納箱 儲物箱 整理盒,private,超大棉麻衣物雙開收納箱儲物箱整理盒,超大 棉麻 衣物 雙開 收納箱 儲物箱 整理盒,ultra-large cotton and linen clothing double o...


In [40]:
test_set.to_csv('./output/1_NMT_test_output2.csv', index=False, encoding='utf_8_sig')

Translate DEV set for BLEU Score Evaluation

In [41]:
dev_set_tcn = pd.read_csv('./preprocessed/cleaned_dev_tcn.csv')

In [42]:
dev_set_tcn

,text,split,product_title_clean,product_title_processed
0,OPPO A75 A75s A73 手机壳 软壳 挂绳壳 大眼兔硅胶壳,private,手机壳软壳挂绳壳大眼兔硅胶壳,手机 壳 软壳 挂绳 壳 大眼 兔 硅胶 壳
1,SOFT 99 鍍膜車蠟(強力撥水型),private,鍍膜車蠟強力撥水型,鍍膜車 蠟 強力 撥水型
2,低糖芒果乾 250g 臻御行,private,低糖芒果乾臻御行,低糖 芒果乾 臻 御行
3,＊小徑文化＊日本進口ROUND TOP space craft - diamond (SC-...,private,小徑文化日本進口,小徑 文化 日本 進口
4,Hello Kitty 凱蒂貓 KITTY 涼鞋 童鞋 白/紅色 小童 no739,private,凱蒂貓涼鞋童鞋白紅色小童,凱蒂貓 涼鞋 童鞋 白紅色 小童
...,...,...,...,...
995,【HippoRed】撕破乐趣★独特风格★中直筒牛仔裤 O591_445,private,撕破乐趣独特风格中直筒牛仔裤,撕破 乐趣 独特 风格 中直 筒 牛仔裤
996,兒童套裝 台灣製薄長袖居家套裝 魔法Baby~k60092,private,兒童套裝台灣製薄長袖居家套裝魔法,兒童 套 裝台 灣 製 薄 長 袖 居家 套裝 魔法
997,LONGCHAMP Le Pliage Neo高密尼龍後背包(中型),private,高密尼龍後背包中型,高密 尼龍 後 背包 中型
998,iFairies 開口可調節戒指★ifairies【56472】【56472】,private,開口可調節戒指,開口 可 調節 戒指


In [43]:
dev_tr_sents_all = []
for i in range(len(dev_set_tcn['product_title_processed'])):
    try: dev_tr_sents_all.append(translate(dev_set_tcn['product_title_processed'][i]))
    except: dev_tr_sents_all.append(np.nan)
        

In [44]:
dev_set_tcn['prediction']= dev_tr_sents_all

In [45]:
dev_set_tcn.drop('split', axis=1, inplace = True)
dev_set_tcn

,text,product_title_clean,product_title_processed,prediction
0,OPPO A75 A75s A73 手机壳 软壳 挂绳壳 大眼兔硅胶壳,手机壳软壳挂绳壳大眼兔硅胶壳,手机 壳 软壳 挂绳 壳 大眼 兔 硅胶 壳,NaN
1,SOFT 99 鍍膜車蠟(強力撥水型),鍍膜車蠟強力撥水型,鍍膜車 蠟 強力 撥水型,NaN
2,低糖芒果乾 250g 臻御行,低糖芒果乾臻御行,低糖 芒果乾 臻 御行,NaN
3,＊小徑文化＊日本進口ROUND TOP space craft - diamond (SC-...,小徑文化日本進口,小徑 文化 日本 進口,small diameter cultural korean version of impo...
4,Hello Kitty 凱蒂貓 KITTY 涼鞋 童鞋 白/紅色 小童 no739,凱蒂貓涼鞋童鞋白紅色小童,凱蒂貓 涼鞋 童鞋 白紅色 小童,katie cat sandals children's shoes white red c...
...,...,...,...,...
995,【HippoRed】撕破乐趣★独特风格★中直筒牛仔裤 O591_445,撕破乐趣独特风格中直筒牛仔裤,撕破 乐趣 独特 风格 中直 筒 牛仔裤,NaN
996,兒童套裝 台灣製薄長袖居家套裝 魔法Baby~k60092,兒童套裝台灣製薄長袖居家套裝魔法,兒童 套 裝台 灣 製 薄 長 袖 居家 套裝 魔法,children's room
997,LONGCHAMP Le Pliage Neo高密尼龍後背包(中型),高密尼龍後背包中型,高密 尼龍 後 背包 中型,high-help magic back seamless
998,iFairies 開口可調節戒指★ifairies【56472】【56472】,開口可調節戒指,開口 可 調節 戒指,open adjustable ring


In [46]:
dev_set_tcn['prediction'] = dev_set_tcn['prediction'].replace(r'^\s*$', np.NaN, regex=True) #removes empty strings or strings containing only spaces with NaN

In [47]:
dev_set_tcn.isna().sum()

text                         0
product_title_clean          6
product_title_processed      6
prediction                 500
dtype: int64

In [48]:
dev_set_tcn.loc[pd.isna(dev_set_tcn).any(1), :].index #check nan after translated rows indexes

Int64Index([  0,   1,   2,   5,   6,   7,   9,  10,  12,  13,
            ...
            977, 980, 981, 982, 984, 987, 989, 992, 994, 995],
           dtype='int64', length=500)

In [49]:
tr_nan_indexes = dev_set_tcn.loc[pd.isna(dev_set_tcn).any(1), :].index #stores nan indexes for later dev_en follow to drop nan rows as well
tr_nan_indexes

Int64Index([  0,   1,   2,   5,   6,   7,   9,  10,  12,  13,
            ...
            977, 980, 981, 982, 984, 987, 989, 992, 994, 995],
           dtype='int64', length=500)

In [50]:
dev_set_tcn =dev_set_tcn.dropna()
dev_set_tcn.isna().sum()

text                       0
product_title_clean        0
product_title_processed    0
prediction                 0
dtype: int64

In [51]:
dev_set_tcn.to_csv('./output/1_NMT_dev_output2.csv',index = False, encoding='utf_8_sig')

In [52]:
dev_set_tcn = pd.read_csv('./output/1_NMT_dev_output2.csv')

In [53]:
dev_set_tcn

,text,product_title_clean,product_title_processed,prediction
0,＊小徑文化＊日本進口ROUND TOP space craft - diamond (SC-...,小徑文化日本進口,小徑 文化 日本 進口,small diameter cultural korean version of impo...
1,Hello Kitty 凱蒂貓 KITTY 涼鞋 童鞋 白/紅色 小童 no739,凱蒂貓涼鞋童鞋白紅色小童,凱蒂貓 涼鞋 童鞋 白紅色 小童,katie cat sandals children's shoes white red c...
2,iFairies 單肩斜背包側背包肩背包【49146】,單肩斜背包側背包肩背包,單肩 斜 背包 側 背包 肩 背包,shoulder diagonal backpack side backpack shoulder
3,Apieu Color Lip Pencil MATT絲緞,絲緞,絲緞,silk satin
4,PAPAGO 368miniHD行車記錄器,行車記錄器,行車 記錄器,driving recorder
...,...,...,...,...
495,加拿大 3 Sprouts 收納箱-兔子 台灣授權代理商,加拿大收納箱兔子台灣授權代理商,加拿大 收納箱 兔子 台灣 授權 代理商,canadian storage box rabbit taiwan agent agent
496,兒童套裝 台灣製薄長袖居家套裝 魔法Baby~k60092,兒童套裝台灣製薄長袖居家套裝魔法,兒童 套 裝台 灣 製 薄 長 袖 居家 套裝 魔法,children's room
497,LONGCHAMP Le Pliage Neo高密尼龍後背包(中型),高密尼龍後背包中型,高密 尼龍 後 背包 中型,high-help magic back seamless
498,iFairies 開口可調節戒指★ifairies【56472】【56472】,開口可調節戒指,開口 可 調節 戒指,open adjustable ring


In [303]:
# !pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [54]:
import sacrebleu

In [55]:
dev_set_en = pd.read_csv('./preprocessed/cleaned_dev_en.csv')

In [56]:
dev_set_en

,translation_output
0,oppo phone case soft rabbit silicone case
1,soft coating car wax strong water watt
2,low sugar mango dry be the royal
3,the culture japan imported round top space cra...
4,hello kitty sandals shoes white red children
...,...
995,hippored torn fun unique style straight jeans
996,kids set table bay thin long sleeve home suit ...
997,longchamp le pliage neo high density nylon bac...
998,ifairies opening adjustable ring ifairies


In [57]:
dev_en_sents = dev_set_en.drop(tr_nan_indexes) # dropping same set of indexes as dev_tr for bleu scorer to match

In [58]:
dev_en_sents

,translation_output
3,the culture japan imported round top space cra...
4,hello kitty sandals shoes white red children
8,ifairies shoulder messenger bag shoulder bag
11,apieu color lip pencil matt silk satin
15,papago hd driving recorder
...,...
993,canada sprouts storage box rabbit taiwan autho...
996,kids set table bay thin long sleeve home suit ...
997,longchamp le pliage neo high density nylon bac...
998,ifairies opening adjustable ring ifairies


In [60]:
refs = [dev_en_sents['translation_output']]
sys = dev_set_tcn['prediction']
bleu = sacrebleu.corpus_bleu(sys, refs, lowercase=True)
print(bleu.precisions)

[44.25967380876239, 19.946707270650933, 9.414519906323186, 4.744744744744745]
